<h1>FaktorII</h1>
Müsterlösung von Woche 4:
Eine Funktion Hensel, der bei Input monische Polynome $f,g,h$ in $\mathbb{Z}[x]$ und eine Primzahl $p$ mit $f=gh \mod p$ und $ggT(g \mod p,h \mod p)=1$ und eine Zahl $k>0$ monische Polynome $u,v$ findet mit $f=uv \mod p^k$ und $u=g \mod p$ und $v=h \mod p$.

In [1]:
def hensel(f,g,h,p,k):
    if k==1:
        return(g,h)
    R=f.parent()
    assert (f-g*h)%p == 0 and f.is_monic() and g.is_monic() and h.is_monic() #Just to check if the input satisfies the assumption.
    gp=g.base_extend(GF(p))
    hp=h.base_extend(GF(p))
    one, a, b = xgcd(hp,gp)
    assert one == 1 #Just to check if the input satisfies the assumption.
    for i in range(1,k):
        #Assume that f=gh mod p^i, find u,v such that f=uv mod p^(i+1), u=g mod p^i, v=h mod p^i
        #First we write f=g*h+p^i*f1
        f1=(f-g*h)//(p^i)
        #Now we need to find g1, h1 such that g1*h+h1*g=f1 
        f1p=f1.base_extend(GF(p))
        g1p=a*f1p
        h1p=b*f1p
        #We need to ensure that deg(g1)<deg(g) (then automatically also deg(h1)<deg(h))
        while g1p.degree() >= gp.degree():
            u=g1p.leading_coefficient()*x^(g1p.degree()-gp.degree())
            g1p=g1p-gp*u
            h1p=h1p+hp*u
        g=g+p^i*R(g1p)
        h=h+p^i*R(h1p)
    return [g,h]
        

Sage hat eine eingebaute Funktion für den LLL-Algorithmus:

In [2]:
M = Matrix(ZZ, [[1,0,10^7],[0,1,788644]])
M.LLL()

[    25   -317   -148]
[   953 -12084  25904]

<h2>Aufgabe a:</h2>
Schreibe eine Funktion faktorII, mit Input:
 <ul>
  <li>$f \in \mathbb{Z}[x]$ monisch, quadratfrei,</li>
  <li>$p$ eine Primzahl, wodurch bei der Berechnung von $ggT(f,f')$ nie geteilt wird (d.h. $p$ teilt die Diskriminante von $f$ nicht),</li>
  <li>$g, h \in \mathbb{Z}[x]$ monisch so dass $(g \mod p) \cdot (h \mod p)=(f \mod p) \in \mathbb{F}_p[x]$, und ausserdem $(h \mod p) \in \mathbb{F}_p[x]$ irreduzibel ist,</li>
</ul> 
Und Output $h_0 \in \mathbb{Z}[x]$ monisch, irreduzibler Teiler von $f$ mit $(h \mod p)|(h_0 \mod p)$ ($h_0$ ist eindeutig).

Sie können die obige Funktionen Hensel und .LLL() (oder Ihre eigene Implementierung davon aus Wochen 4 und 6) verwenden. 

In [114]:
def faktorII(f,p,h,g):
    n = f.degree()
    l = h.degree()
    
    if l >= n:
        raise ArgumentError(f"Degree of f (= {n}) must be larger than degree of l (= {l})")
    
    m = l - 1
    
    while True:
        m += 1
        
        # We now want a k such that:
        # p^{k*l} > 2^{m*n/2} * binom(2m, m)^{n/2} * ||f||^{m+n}
        # Thus we simply calculate the RHS, take the log with base p, divide by l, and round up. Easy!
        rhs = 2^(m * n / 2) * binomial(2*m, m)^(n/2) * f.norm(2)^(m+n)
        k = ceil(log(rhs, p) / l)
        assert(p^(k*l) > rhs)
        
        # Now we want a polynomial ~h over the integers, such that:
        #   ~h \equiv h mod p
        #   ~h mod p^k | f mod p^k
        # Which we'll do with the Hensel elevator.
        # Hensel will, given f, g, h, p, k, return polyonmials u, v, with:
        #   u \equiv g mod p
        #   v \equiv h mod p
        #   f \equiv uv mod p^k
        # Thus, v will be a fitting choice for ~h.
        u, v = hensel(f, g, h, p, k)
                
        # We now construct a basis of a lattice L_{m, k} in R[x]_{<= m}, using:
        #   p^k * x^i for i = 0 to l-1
        #   x^i * ~h  for i = 0 to m-l
        # (For a total of m basis vectors)
        l_basis = Matrix(ZZ, m, m)
        for i in range(0, l): # [0, l-1]
            # This basis vector will represent p^k * x^i, so the all-zero vector with p^k in the (i+1)-th field
            b = vector(ZZ, m)
            b[i] = p^k
            l_basis[i] = b
            
        for i in range(0, m-l): # [0, m-l-1]
            # This basis vector will represent x^i * ~h, so ~h "shifted right" by i places
            b = vector(ZZ, m)
            b[i : v.degree() + 1 + i] = v # Length of coefficient vector of v is deg(v) + 1
            l_basis[i + l] = b
        
        print(l_basis, "\n")
        
        # Calculate 3/4-reduced LLL basis
        reduced_basis = l_basis.LLL(delta=3/4)
        
        # We are done when ||p_1|| < (p^(kl) / ||f||^m)^(1/n)
        # where p_1 is the first basis vector of the reduced basis
        threshold = (p^(k*l) / (f.norm(2)^m))^(1/n)
        if reduced_basis[0].norm(2) < threshold:
            return reduced_basis[0]  

In [115]:
#Test
R.<x> = PolynomialRing(ZZ)
f=x^6 + 3*x^5 + x^4 + x^2 + 3*x + 1
p=3
print(f.factor())
print(f.factor_mod(p))

print(faktorII(f,p,(x^2 + 1),(x^2 + x + 2) * (x^2 + 2*x + 2))) #Wanted output: x^2 + 3*x + 1
print(faktorII(f,p,(x^2 + x + 2),(x^2 + 1) * (x^2 + 2*x + 2))) #Wanted output: x^4 + 1

(x^2 + 3*x + 1) * (x^4 + 1)
(x^2 + 1) * (x^2 + x + 2) * (x^2 + 2*x + 2)
[59049     0]
[    0 59049] 

[4782969       0       0]
[      0 4782969       0]
[      1       3       1] 

(1, 3, 1)
[59049     0]
[    0 59049] 

[4782969       0       0]
[      0 4782969       0]
[4782968 4077076       1] 

[129140163         0         0         0]
[        0 129140163         0         0]
[129140162  18425983         1         0]
[        0 129140162  18425983         1] 

[10460353203           0           0           0           0]
[          0 10460353203           0           0           0]
[10460353202  7508555437           1           0           0]
[          0 10460353202  7508555437           1           0]
[          0           0 10460353202  7508555437           1] 

(1, 0, 0, 0, 1)
